<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning

This notebook should be run in a Watson Studio project, using **IBM Runtime 23.1 on Python 3.10 XS** runtime environment. **If you are viewing this in Watson Studio and do not see the required runtime env in the upper right corner of your screen, please update the runtime now.** It requires service credentials for the following Cloud services:
  * Watson OpenScale 
  * Watson Machine Learning 
  * Cloud Object Storage
  
If you have a paid Cloud account, you may also provision a **Databases for PostgreSQL** or **Db2 Warehouse** service to take full advantage of integration with Watson Studio. If you choose not to provision this paid service, you can use the free internal PostgreSQL storage with OpenScale, but will not be able to configure continuous learning for your model.

The notebook will train, create and deploy a German Credit Risk model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

### Contents

- [Setup](#setup)
- [Model building and deployment](#model)
- [OpenScale configuration](#openscale)
- [Quality monitor and feedback logging](#quality)
- [Fairness,Drift monitoring and explanations](#fairness)
- [Custom monitors and metrics](#custom)
- [Historical data](#historical)

### Note: Samples using most latest openscale V2 client from public pypi. It does not cover followings aspects for now:

- Historical payload logging 
- Historical manual labeling

# Setup <a name="setup"></a>

## Package installation
Make sure you have ibm-watson-openscale package version greater than 3.0.17

In [1]:
!pip list | grep ibm-watson-openscale

ibm-watson-openscale                    3.0.39

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [ ]:
# !rm -rf /home/spark/shared/user-libs/python3.7*

# If you are executing this notebook in non IBM Watson Studio env then uncomment the pip install statements
#!pip install --upgrade scikit-learn==1.1 --user --no-cache | tail -n 1  
# !pip install --upgrade pandas==0.25.3 --user --no-cache | tail -n 1
# !pip install --upgrade requests==2.23 --user --no-cache | tail -n 1
# !pip install --upgrade numpy==1.20.3 --user --no-cache | tail -n 1
# !pip install --upgrade SciPy --user --no-cache | tail -n 1

!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade "ibm-watson-openscale~=3.0.34" --user --no-cache | tail -n 1

## Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key, generate an IAM token using that key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [2]:
CLOUD_API_KEY = "***"
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

### WML credentials example with API key

In [3]:
WML_CREDENTIALS = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey": CLOUD_API_KEY
            }

This tutorial can use Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL to create a datamart for OpenScale.

If you have previously configured OpenScale, it will use your existing datamart, and not interfere with any models you are currently monitoring. Do not update the cell below.

If you do not have a paid Cloud account or would prefer not to provision this paid service, you may use the free internal PostgreSQL service with OpenScale. Do not update the cell below.

To provision a new instance of Db2 Warehouse, locate [Db2 Warehouse in the Cloud catalog](https://cloud.ibm.com/catalog/services/db2-warehouse), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Db2 Warehouse credentials into the cell below.

To provision a new instance of Databases for PostgreSQL, locate [Databases for PostgreSQL in the Cloud catalog](https://cloud.ibm.com/catalog/services/databases-for-postgresql), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Databases for PostgreSQL credentials into the cell below.

In [4]:
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}
DB_CREDENTIALS = None
SCHEMA_NAME = None

__If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service.__ If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the __KEEP_MY_INTERNAL_POSTGRES__ variable below to __False__ below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. __*NO DATA MIGRATION WILL OCCUR.*__

In [5]:
KEEP_MY_INTERNAL_POSTGRES = True

In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit [getting started with COS tutorial](https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-getting-started). 
You can find `COS_API_KEY_ID` and `COS_RESOURCE_CRN` variables in **_Service Credentials_** in menu of your COS instance. Used COS Service Credentials must be created with _Role_ parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription.  
`COS_ENDPOINT` variable can be found in **_Endpoint_** field of the menu.

In [6]:
COS_API_KEY_ID = "***"
COS_RESOURCE_CRN = "***" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [7]:
BUCKET_NAME = "" #example: "credit-risk-training-data"

## Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Model building and deployment <a name="model"></a>

In this section you will learn how to train Scikit-learn model and next deploy it as web-service using Watson Machine Learning service.

## Load the training data from github

In [8]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_data_biased_training.csv -O german_credit_data_biased_training.csv

--2024-07-26 12:48:31--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

german_credit_data_ 100%[===================>] 673.46K  --.-KB/s    in 0.1s    

2024-07-26 12:48:32 (5.28 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]



In [9]:
import numpy as np
import pandas as pd

training_data_file_name = "german_credit_data_biased_training.csv"
data_df = pd.read_csv(training_data_file_name)

## Explore data

In [10]:
data_df.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


In [10]:
print('Columns: ', list(data_df.columns))
print('Number of columns: ', len(data_df.columns))

Columns:  ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'Risk']
Number of columns:  21


As you can see, the data contains twenty one fields. `Risk` field is the one you would like to predict using feedback data.

In [11]:
print('Number of records: ', data_df.Risk.count())

Number of records:  5000


In [12]:
target_count = data_df.groupby('Risk')['Risk'].count()
target_count

Risk
No Risk    3330
Risk       1670
Name: Risk, dtype: int64

## Visualize data

In [13]:
#target_count.plot.pie(figsize=(8, 8));

## Save training data to Cloud Object Storage

In [14]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [15]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

## Create a model
In this section you will learn how to:

- Prepare data for training a model
- Create machine learning pipeline
- Train a model

In [16]:
MODEL_NAME = "Scikit German Risk Model WML V4"
DEPLOYMENT_NAME = "Scikit German Risk Deployment WML V4"

### You will start with importing required libraries

In [17]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

### Splitting the data into train and test

In [18]:
train_data, test_data = train_test_split(data_df, test_size=0.2)

### Preparing the pipeline

In [19]:
features_idx = np.s_[0:-1]
all_records_idx = np.s_[:]
first_record_idx = np.s_[0]

In this step you will encode target column labels into numeric values. You can use `inverse_transform` to decode numeric predictions into labels.

In [20]:
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]
ct = ColumnTransformer([("ohe", OneHotEncoder(), list(np.array(train_data.columns)[features_idx][string_fields]))])
clf_linear = SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, tol=1e-5)

pipeline_linear = Pipeline([('ct', ct), ('clf_linear', clf_linear)])

### Train a model

In [21]:
risk_model = pipeline_linear.fit(train_data.drop('Risk', axis=1), train_data.Risk)

### Evaluate the model

In [22]:
from sklearn.metrics import roc_auc_score

predictions = risk_model.predict(test_data.drop('Risk', axis=1))
indexed_preds = [0 if prediction=='No Risk' else 1 for prediction in predictions]

real_observations = test_data.Risk.replace('Risk', 1)
real_observations = real_observations.replace('No Risk', 0).values

auc = roc_auc_score(real_observations, indexed_preds)
print(auc)

0.7553885422575156


/var/folders/5f/j8f1jffd09l7lf7gq61x4h8m0000gn/T/ipykernel_77787/1678086252.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  real_observations = real_observations.replace('No Risk', 0).values


## Publish the model

In this section, the notebook uses the supplied Watson Machine Learning credentials to save the model (including the pipeline) to the WML instance. Previous versions of the model are removed so that the notebook can be run again, resetting all data for another demo.

In [23]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.360'

### Listing all the available spaces

In [24]:
wml_client.spaces.list(limit=10)

------------------------------------  -------------------------------------------------------------------  ------------------------
ID                                    NAME                                                                 CREATED
be45ab4c-1fb7-440c-9b03-2909067e45e0  Automotive Demo - Quality report summarization                       2024-05-27T13:13:27.233Z
e0ee6250-7ef6-42c3-8ffa-350d9b0df578  pre-prod-space                                                       2024-02-28T07:35:30.368Z
63c5982f-7160-41c4-86f7-1310a8ab32cb  prompt-space                                                         2024-01-15T19:29:21.535Z
f04e0e73-a1b7-4ae9-a08d-7e16add4fe08  llm. space                                                           2023-11-24T13:57:05.167Z
d1afbea3-e899-4ed3-b9a6-0686751508c3  wml                                                                  2023-09-21T07:06:42.577Z
6f7c3969-6d3f-4f9a-b97a-b534f4e4fef3  AutoAIDemo                                             

,ID,NAME,CREATED
0,be45ab4c-1fb7-440c-9b03-2909067e45e0,Automotive Demo - Quality report summarization,2024-05-27T13:13:27.233Z
1,e0ee6250-7ef6-42c3-8ffa-350d9b0df578,pre-prod-space,2024-02-28T07:35:30.368Z
2,63c5982f-7160-41c4-86f7-1310a8ab32cb,prompt-space,2024-01-15T19:29:21.535Z
3,f04e0e73-a1b7-4ae9-a08d-7e16add4fe08,llm. space,2023-11-24T13:57:05.167Z
4,d1afbea3-e899-4ed3-b9a6-0686751508c3,wml,2023-09-21T07:06:42.577Z
5,6f7c3969-6d3f-4f9a-b97a-b534f4e4fef3,AutoAIDemo,2023-08-25T02:50:27.113Z
6,0b7992c2-3991-4145-a5ba-d5b428261171,openscale-express-path-preprod-80e6093f-5acf-4...,2023-08-16T06:59:18.538Z
7,3226c381-5ae0-4bc4-b306-fc638c785e47,openscale-express-path-80e6093f-5acf-4eb7-9da6...,2023-08-16T06:58:57.332Z


In [25]:
WML_SPACE_ID='***' # use space id here
wml_client.set.default_space(WML_SPACE_ID)

'SUCCESS'

### Remove existing model and deployment

In [26]:

deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

{'entity': {'asset': {'id': '8680d72c-91b3-4a66-b4c4-0d8696555e57'}, 'custom': {}, 'deployed_asset_type': 'model', 'name': 'Scikit German Risk Deployment WML V4', 'online': {}, 'space_id': 'd1afbea3-e899-4ed3-b9a6-0686751508c3', 'status': {'failure': {'trace': 'NO_TRACE_AVAILABLE', 'errors': [{'code': '65f12c527e3ebdfa572e5909daa0a9b2', 'message': 'Model loading through joblib failed with exception: KeyError and message: 54. The model object provided for deployment is either invalid or trained using a version of scikit-learn other than: 1.1'}]}, 'inference': [{'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/002ee049-b7ec-4374-9a4f-7760d005cb3e/predictions'}], 'online_url': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/002ee049-b7ec-4374-9a4f-7760d005cb3e/predictions'}, 'serving_urls': ['https://us-south.ml.cloud.ibm.com/ml/v4/deployments/002ee049-b7ec-4374-9a4f-7760d005cb3e/predictions'], 'state': 'failed'}}, 'metadata': {'created_at': '2024-07-26T07:19:43.93

In [27]:
datasource_type = wml_client.connections.get_datasource_type_uid_by_name('bluemixcloudobjectstorage')
conn_meta_props= {
    wml_client.connections.ConfigurationMetaNames.NAME: "Connection My COS ",
    wml_client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: datasource_type,
    wml_client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection to my COS",
    wml_client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': BUCKET_NAME,
        'api_key': COS_API_KEY_ID,
        'resource_instance_id': COS_RESOURCE_CRN,
        'iam_url': "https://iam.ng.bluemix.net/oidc/token",
        'url': COS_ENDPOINT
    }
}

conn_details = wml_client.connections.create(meta_props=conn_meta_props)
connection_id = wml_client.connections.get_uid(conn_details)

training_data_references = [
    {
        "id": "German Credit Risk", 
        "type": "connection_asset",
        "connection": {
            "id": connection_id,
            "href": "/v2/connections/" + connection_id + "?space_id=" + WML_SPACE_ID

        },
        "location": {
            "bucket": BUCKET_NAME,
            "file_name": training_data_file_name
        }
    }    
]

Creating connections...
SUCCESS


In [28]:
#Note if there is specification related exception or specification ID is None then use "default_py3.8" instead of "default_py3.7_opence"
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-23.1-py3.10")
print("Software Specification ID: {}".format(software_spec_uid))

model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:"{}".format(MODEL_NAME),
        wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.1",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        #wml_client._models.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "Risk",
    }

Software Specification ID: 336b29df-e0e1-5e7d-b6a5-f6ab722625b2


In [29]:
print("Storing model ...")

published_model_details = wml_client.repository.store_model(model=risk_model, meta_props=model_props, training_data=data_df.drop(["Risk"], axis=1), training_target=data_df.Risk)
model_uid = wml_client.repository.get_model_id(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

Storing model ...
Done
Model ID: 8680d72c-91b3-4a66-b4c4-0d8696555e57


## Deploy the model

The next section of the notebook deploys the model as a RESTful web service in Watson Machine Learning. The deployed model will have a scoring URL you can use to send data to the model for predictions.

In [30]:
print("Deploying model...")
deployment_details = wml_client.deployments.create(
    model_uid, 
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(DEPLOYMENT_NAME),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
deployment_uid=wml_client.deployments.get_uid(deployment_details)

print("Scoring URL:" + scoring_url)
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))



#######################################################################################

Synchronous deployment creation for uid: 'ce0b71a4-ecee-4e0d-b51a-123e0fad4acd' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='db664cd4-2467-4975-b314-e24115c37ea5'
------------------------------------------------------------------------------------------------


Scoring URL:https://us-south.ml.cloud.ibm.com/ml/v4/deployments/db664cd4-2467-4975-b314-e24115c37ea5/predictions
Model id: ce0b71a4-ecee-4e0d-b51a-123e0fad4acd
Deployment id: db664cd4-2467-4975-b314-e24115c37ea5


## Score the model

In [31]:
scoring_url = "https://us-south.ml.cloud.ibm.com/ml/v4/deployments/db664cd4-2467-4975-b314-e24115c37ea5/predictions"
model_uid = "ce0b71a4-ecee-4e0d-b51a-123e0fad4acd"
deployment_uid = "db664cd4-2467-4975-b314-e24115c37ea5"
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
                  "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
                  "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
                  "Telephone", "ForeignWorker"]
values = [
            ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
             "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
             4, "savings_insurance", 36, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
        ]

scoring_payload = {"input_data": [{"fields": fields, "values": values}]}

In [32]:
predictions = wml_client.deployments.score(deployment_uid, scoring_payload)
predictions

{'predictions': [{'fields': ['CheckingStatus',
    'LoanDuration',
    'CreditHistory',
    'LoanPurpose',
    'LoanAmount',
    'ExistingSavings',
    'EmploymentDuration',
    'InstallmentPercent',
    'Sex',
    'OthersOnLoan',
    'CurrentResidenceDuration',
    'OwnsProperty',
    'Age',
    'InstallmentPlans',
    'Housing',
    'ExistingCreditsCount',
    'Job',
    'Dependents',
    'Telephone',
    'ForeignWorker',
    'CheckingStatus_IX',
    'CreditHistory_IX',
    'LoanPurpose_IX',
    'ExistingSavings_IX',
    'EmploymentDuration_IX',
    'Sex_IX',
    'OthersOnLoan_IX',
    'OwnsProperty_IX',
    'InstallmentPlans_IX',
    'Housing_IX',
    'Job_IX',
    'Telephone_IX',
    'ForeignWorker_IX',
    'features',
    'rawPrediction',
    'probability',
    'prediction',
    'predictedLabel'],
   'values': [['no_checking',
     13,
     'credits_paid_to_date',
     'car_new',
     1343,
     '100_to_500',
     '1_to_4',
     2,
     'female',
     'none',
     3,
     'savings

# Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [33]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment this line if using IAM token as authenticator
wos_client = APIClient(authenticator=authenticator)
wos_client.version

'3.0.39'

## Create schema and datamart

### Set up datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were **not** supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created there **unless** there is an existing datamart **and** the **KEEP_MY_INTERNAL_POSTGRES** variable is set to **True**. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the German Credit model will be removed from OpenScale monitoring.

In [34]:
wos_client.data_marts.show()

AIOSFASTPATH-80E6093F-5ACF-4EB7-9DA6-7BA9BF56A929,None,True,active,2024-05-16 06:09:07.089000+00:00,80e6093f-5acf-4eb7-9da6-7ba9bf56a929


In [35]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 80e6093f-5acf-4eb7-9da6-7ba9bf56a929


### Remove existing service provider connected with used  WML instance. 

Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one. 

In [36]:
SERVICE_PROVIDER_NAME = "Watson Machine Learning V2"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook."

In [37]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

Deleted existing service_provider for WML instance: e13bfbbb-e639-4875-bbd5-6d2de8e834d0


## Add service provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

**Note:** You can bind more than one engine instance if needed by calling `wos_client.service_providers.add` method. Next, you can refer to particular service provider using `service_provider_id`.

In [38]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_SPACE_ID,
        operational_space_id = "production",
        credentials=WMLCredentialsCloud(
            apikey=CLOUD_API_KEY,      ## use `apikey=IAM_TOKEN` if using IAM_TOKEN
            url=WML_CREDENTIALS["url"],
            instance_id=None
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of monitoring run 29490afc-07e2-8fbb-be08-g8aic85da02d 




finished

---------------------------
 Successfully finished run 
---------------------------




In [39]:
wos_client.service_providers.show()

89cbf463-87d9-4032-9c0c-e371dd68d156,active,Watson Machine Learning V2,watson_machine_learning,2024-07-26 07:36:57.982000+00:00,a64a04b8-8164-4efc-b7b4-47c0104a6215
None,active,OpenScale Headless Service Provider-3,custom_machine_learning,2024-07-22 08:09:30.914000+00:00,3072a089-1573-4244-b158-024fe4dfda21
89cbf463-87d9-4032-9c0c-e371dd68d156,active,Watson Machine Learning V2-2,watson_machine_learning,2024-07-16 06:11:13.858000+00:00,6a38b72c-4aae-4fb0-b786-9b9edacf64bf
None,active,Azure Machine Learning,azure_machine_learning,2024-07-15 11:18:22.795000+00:00,db2618f1-ef2b-4251-a22c-bd347702672d
None,active,AWS Machine Learning,amazon_sagemaker,2024-07-11 09:29:39.272000+00:00,4fa5625b-5401-4a31-a423-d480ea184cda
89cbf463-87d9-4032-9c0c-e371dd68d156,active,aWatson Machine Learning V2_amzon,watson_machine_learning,2024-06-06 06:03:03.459000+00:00,84265de0-8f4b-4257-8743-7c2525e86b00
89cbf463-87d9-4032-9c0c-e371dd68d156,active,WOS ExpressPath WML pre_production binding,watson_machine_learning,2024-05-16 06:09:28.464000+00:00,19fc94cc-9ed2-4ac6-a98e-1acf1a9e729e
89cbf463-87d9-4032-9c0c-e371dd68d156,active,WOS ExpressPath WML production binding,watson_machine_learning,2024-05-16 06:09:12.960000+00:00,7ef8cc5f-65b1-4f83-9d1b-d2d508420958


## Subscriptions

### Remove existing credit risk subscriptions

In [40]:
wos_client.subscriptions.show()

8a4bcb3a-f280-4970-9185-d71e7b3f1e17,model,[asset] GCR Headless Subscription,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,eff79a1b-5c31-4d28-bb57-3db70cdd2bd7,[asset] GCR Headless Subscription,3072a089-1573-4244-b158-024fe4dfda21,active,2024-07-22 08:09:37.221000+00:00,9df07f22-a1c5-4e6b-9e19-493e98dc439c
085460ef94636166aea5800e9ea26168,model,GermanCreditRisk.2019.1.9.10.41.58.611,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,563f01d37f720857b95c557dc76176ad,GermanCreditRisk.2019.1.9.10.41.58.611,db2618f1-ef2b-4251-a22c-bd347702672d,active,2024-07-15 11:20:20.209000+00:00,1ae2f751-e220-48a9-b049-d2404311f314
a7fc7ed9-f000-4f27-988a-54fbc8e397e6,model,GermanCreditRiskModelChallenger,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,38059feb-e4c5-4037-90ea-a18f77a8ec5b,GermanCreditRiskModelChallenger,19fc94cc-9ed2-4ac6-a98e-1acf1a9e729e,active,2024-05-16 06:09:55.832000+00:00,81044f9c-27a8-42f2-90be-9cef8b33fe39


This code removes previous subscriptions to the German Credit model to refresh the monitors with the new model and new data.

In [41]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print("Deleted existing subscription for model", sub_model_id)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [42]:
ASSET_NAME = "Scikit German Risk Model WML V4"
asset = Asset(
    asset_id=model_uid,
    name = ASSET_NAME,
    url=deployment["entity"]["status"]["online_url"]["url"],
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION
)
asset_deployment = AssetDeploymentRequest(
    deployment_id=deployment_uid,
    name=DEPLOYMENT_NAME,
    deployment_type=DeploymentTypes.ONLINE,
    url=deployment["entity"]["status"]["online_url"]["url"]
)
training_data_reference = TrainingDataReference(
    type="cos",
    location=COSTrainingDataReferenceLocation(
        bucket=BUCKET_NAME,
        file_name=training_data_file_name
    ),
    connection=COSTrainingDataReferenceConnection.from_dict(
        {
            "resource_instance_id": COS_RESOURCE_CRN,
            "url": COS_ENDPOINT,
            "api_key": COS_API_KEY_ID,
            "iam_url": "https://iam.bluemix.net/oidc/token"
        }
    )
)
asset_properties_request = AssetPropertiesRequest(
    label_column="Risk",
    probability_fields=["probability"],
    prediction_field="prediction",
    feature_fields=["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_fields=["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
    training_data_reference=training_data_reference
    ,training_data_schema=SparkStruct(
    type="struct",
    fields=training_fields
)
)

In [43]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=asset,
        deployment=asset_deployment,
        asset_properties=asset_properties_request,background_mode = False).result
subscription_id = subscription_details.metadata.id
print(subscription_details)




 Waiting for end of adding subscription 61a97558-9430-4cc5-a9c9-7d44d1caeb27 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------


{
  "metadata": {
    "id": "61a97558-9430-4cc5-a9c9-7d44d1caeb27",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:subscription:61a97558-9430-4cc5-a9c9-7d44d1caeb27",
    "url": "/v2/subscriptions/61a97558-9430-4cc5-a9c9-7d44d1caeb27",
    "created_at": "2024-07-26T07:41:53.116000Z",
    "created_by": "IBMid-662005298W",
    "modified_at": "2024-07-26T07:41:54.957000Z",
    "modified_by": "IBMid-662005298W"
  },
  "entity": {
    "data_mart_id": "80e6093f-5acf-4eb7-9da6-7ba9bf56a929",
    "service_provider_id": "a64a04b8-8164-4efc-b7b4-47c0104a6215",
    "asset": {
      "asset_id": "ce0b71a4-ecee-4e0d-b51a-123e0fad4acd",
      "url": "https://us-south.ml.cloud.ibm.com/ml/v4/deploymen

In [44]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

Payload data set id: 8e6a7797-72ae-4019-923c-0e472d680b20


In [45]:
wos_client.data_sets.show()

80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,61a97558-9430-4cc5-a9c9-7d44d1caeb27,subscription,model_health,2024-07-26 07:41:56.598000+00:00,5a417a0f-70e4-4081-ba00-21d9628fa234
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,61a97558-9430-4cc5-a9c9-7d44d1caeb27,subscription,payload_logging_error,2024-07-26 07:41:54.815000+00:00,7a69d49e-f3c5-43f9-947c-249758fb0cb1
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,61a97558-9430-4cc5-a9c9-7d44d1caeb27,subscription,manual_labeling,2024-07-26 07:41:54.588000+00:00,88728098-4f6b-4735-9eba-4e1b0b4c7bae
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,61a97558-9430-4cc5-a9c9-7d44d1caeb27,subscription,payload_logging,2024-07-26 07:41:54.453000+00:00,8e6a7797-72ae-4019-923c-0e472d680b20
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,9df07f22-a1c5-4e6b-9e19-493e98dc439c,subscription,drift_stats,2024-07-23 02:22:08.780000+00:00,20b9b31e-2f9a-4d6d-b46e-59e229ac2814
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,9df07f22-a1c5-4e6b-9e19-493e98dc439c,subscription,drift_insights,2024-07-23 02:22:08.785000+00:00,d7d34ba4-d0b5-44d9-8920-35bad12892d9
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,9df07f22-a1c5-4e6b-9e19-493e98dc439c,subscription,drift_intervals,2024-07-23 02:22:08.892000+00:00,3612da76-5505-4c2c-a25d-d8a1a6f06497
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,9df07f22-a1c5-4e6b-9e19-493e98dc439c,subscription,drift_scores,2024-07-23 02:22:08.791000+00:00,870f9ea8-a5e2-4ddc-bd1d-92bcc66bf56f
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,9df07f22-a1c5-4e6b-9e19-493e98dc439c,subscription,feedback,2024-07-22 08:16:28.816000+00:00,a09f470c-dbec-40ca-ac76-1bd285d9a609
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,9df07f22-a1c5-4e6b-9e19-493e98dc439c,subscription,explanations,2024-07-22 08:10:42.504000+00:00,d73cd189-e4d2-467f-b4f1-5215ceb9d708


Note: First 10 records were displayed.




Get subscription list

In [46]:
wos_client.subscriptions.show()

ce0b71a4-ecee-4e0d-b51a-123e0fad4acd,model,Scikit German Risk Model WML V4,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,db664cd4-2467-4975-b314-e24115c37ea5,Scikit German Risk Deployment WML V4,a64a04b8-8164-4efc-b7b4-47c0104a6215,active,2024-07-26 07:41:53.116000+00:00,61a97558-9430-4cc5-a9c9-7d44d1caeb27
8a4bcb3a-f280-4970-9185-d71e7b3f1e17,model,[asset] GCR Headless Subscription,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,eff79a1b-5c31-4d28-bb57-3db70cdd2bd7,[asset] GCR Headless Subscription,3072a089-1573-4244-b158-024fe4dfda21,active,2024-07-22 08:09:37.221000+00:00,9df07f22-a1c5-4e6b-9e19-493e98dc439c
085460ef94636166aea5800e9ea26168,model,GermanCreditRisk.2019.1.9.10.41.58.611,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,563f01d37f720857b95c557dc76176ad,GermanCreditRisk.2019.1.9.10.41.58.611,db2618f1-ef2b-4251-a22c-bd347702672d,active,2024-07-15 11:20:20.209000+00:00,1ae2f751-e220-48a9-b049-d2404311f314
a7fc7ed9-f000-4f27-988a-54fbc8e397e6,model,GermanCreditRiskModelChallenger,80e6093f-5acf-4eb7-9da6-7ba9bf56a929,38059feb-e4c5-4037-90ea-a18f77a8ec5b,GermanCreditRiskModelChallenger,19fc94cc-9ed2-4ac6-a98e-1acf1a9e729e,active,2024-05-16 06:09:55.832000+00:00,81044f9c-27a8-42f2-90be-9cef8b33fe39


### Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. The sends a few records for predictions.

In [47]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",10,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",20,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",24,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",17,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",66,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",14,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",35,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",23,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
predictions = wml_client.deployments.score(deployment_uid, payload_scoring)

print("Single record scoring result:", "\n fields:", predictions["predictions"][0]["fields"], "\n values: ", predictions["predictions"][0]["values"][0])

Single record scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'LoanPurpose_IX', 'ExistingSavings_IX', 'EmploymentDuration_IX', 'Sex_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'InstallmentPlans_IX', 'Housing_IX', 'Job_IX', 'Telephone_IX', 'ForeignWorker_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['no_checking', 10, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, [20, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19], [1.0, 1.0, 1.0, 10.0, 

## Check if WML payload logging worked else manually store payload records

In [48]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": predictions['predictions'][0]['fields'], "values":predictions['predictions'][0]['values']},
                   response_time=460
               )],background_mode = False)
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

Number of records in the payload logging table: 100


# Quality monitoring and feedback logging <a name="quality"></a>

## Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [49]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation f16dff88-1305-4584-8c7d-3056728e79a4 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [50]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'f16dff88-1305-4584-8c7d-3056728e79a4'

## Feedback logging

The code below downloads and stores enough feedback data to meet the minimum threshold so that OpenScale can calculate a new accuracy measurement. It then kicks off the accuracy monitor. The monitors run hourly, or can be initiated via the Python API, the REST API, or the graphical user interface.

In [51]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json

--2024-07-31 14:08:41--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50890 (50K) [text/plain]
Saving to: ‘additional_feedback_data_v2.json’

additional_feedback 100%[===================>]  49.70K  --.-KB/s    in 0.04s   

2024-07-31 14:08:42 (1.36 MB/s) - ‘additional_feedback_data_v2.json’ saved [50890/50890]



### Get feedback logging dataset ID

In [52]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

{
  "data_sets": [
    {
      "metadata": {
        "id": "3b44db5f-e7d0-4e57-a4b2-8d6622b23be7",
        "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/181ed6cc388f47bd9d862fe066f9cfce:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:data_set:3b44db5f-e7d0-4e57-a4b2-8d6622b23be7",
        "url": "/v2/data_sets/3b44db5f-e7d0-4e57-a4b2-8d6622b23be7",
        "created_at": "2024-07-16T06:12:30.424000Z",
        "created_by": "iam-ServiceId-2e5c9fda-38bf-4279-9712-cdb3b6f3a7ad",
        "modified_at": "2024-07-16T06:12:30.693000Z",
        "modified_by": "iam-ServiceId-2e5c9fda-38bf-4279-9712-cdb3b6f3a7ad"
      },
      "entity": {
        "data_mart_id": "80e6093f-5acf-4eb7-9da6-7ba9bf56a929",
        "name": "f1a960ef-b474-43f9-8621-d0b1f2a4ef68_feedback",
        "description": "f1a960ef-b474-43f9-8621-d0b1f2a4ef68_feedback",
        "type": "feedback",
        "target": {
          "target_type": "subscription",
          "target_id": "f1a960ef-b474-43f9-8621-d0b1f2a4ef68"
        },
    

In [53]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)

In [54]:
wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False)




 Waiting for end of storing records with request id: 7a62e26f-1e79-4253-bf9c-857e1b54d00b 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [55]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

98

In [56]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 71790afc-05d2-4f4b-be08-f0afc85da02d 




finished

---------------------------
 Successfully finished run 
---------------------------




In [57]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2024-07-16 06:13:19.969000+00:00,true_positive_rate,1267b942-5fe0-423c-8263-b32816c30099,0.36363636363636365,None,None,['model_type:original'],quality,f16dff88-1305-4584-8c7d-3056728e79a4,92b130ed-fc31-43c2-8b0e-a5f8d3a91345,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:13:19.969000+00:00,area_under_roc,1267b942-5fe0-423c-8263-b32816c30099,0.6587412587412588,0.8,None,['model_type:original'],quality,f16dff88-1305-4584-8c7d-3056728e79a4,92b130ed-fc31-43c2-8b0e-a5f8d3a91345,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:13:19.969000+00:00,precision,1267b942-5fe0-423c-8263-b32816c30099,0.8,None,None,['model_type:original'],quality,f16dff88-1305-4584-8c7d-3056728e79a4,92b130ed-fc31-43c2-8b0e-a5f8d3a91345,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:13:19.969000+00:00,matthews_correlation_coefficient,1267b942-5fe0-423c-8263-b32816c30099,0.41672426371926674,None,None,['model_type:original'],quality,f16dff88-1305-4584-8c7d-3056728e79a4,92b130ed-fc31-43c2-8b0e-a5f8d3a91345,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:13:19.969000+00:00,f1_measure,1267b942-5fe0-423c-8263-b32816c30099,0.5000000000000001,None,None,['model_type:original'],quality,f16dff88-1305-4584-8c7d-3056728e79a4,92b130ed-fc31-43c2-8b0e-a5f8d3a91345,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:13:19.969000+00:00,accuracy,1267b942-5fe0-423c-8263-b32816c30099,0.7551020408163265,None,None,['model_type:original'],quality,f16dff88-1305-4584-8c7d-3056728e79a4,92b130ed-fc31-43c2-8b0e-a5f8d3a91345,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:13:19.969000+00:00,label_skew,1267b942-5fe0-423c-8263-b32816c30099,0.6909336273400493,None,None,['model_type:original'],quality,f16dff88-1305-4584-8c7d-3056728e79a4,92b130ed-fc31-43c2-8b0e-a5f8d3a91345,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:13:19.969000+00:00,gini_coefficient,1267b942-5fe0-423c-8263-b32816c30099,0.31748251748251755,None,None,['model_type:original'],quality,f16dff88-1305-4584-8c7d-3056728e79a4,92b130ed-fc31-43c2-8b0e-a5f8d3a91345,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:13:19.969000+00:00,log_loss,1267b942-5fe0-423c-8263-b32816c30099,0.4493805793027406,None,None,['model_type:original'],quality,f16dff88-1305-4584-8c7d-3056728e79a4,92b130ed-fc31-43c2-8b0e-a5f8d3a91345,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:13:19.969000+00:00,false_positive_rate,1267b942-5fe0-423c-8263-b32816c30099,0.046153846153846156,None,None,['model_type:original'],quality,f16dff88-1305-4584-8c7d-3056728e79a4,92b130ed-fc31-43c2-8b0e-a5f8d3a91345,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68


Note: First 10 records were displayed.


# Fairness monitoring and explanations <a name="fairness"></a>

### Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:
  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 200 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [58]:
wos_client.monitor_instances.show()

80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,f1a960ef-b474-43f9-8621-d0b1f2a4ef68,subscription,model_health,2024-07-16 06:11:34.218000+00:00,bb904906-53a8-425b-8831-d6f8b499b18a
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,f1a960ef-b474-43f9-8621-d0b1f2a4ef68,subscription,quality,2024-07-16 06:12:29.680000+00:00,f16dff88-1305-4584-8c7d-3056728e79a4
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,f1a960ef-b474-43f9-8621-d0b1f2a4ef68,subscription,performance,2024-07-16 06:11:36.282000+00:00,ab36daa4-b83e-42a4-9a81-b4cb66b1b8ec
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,345f4646-871a-4e0c-b30f-abcf9a83ea71,subscription,mrm,2024-07-05 14:08:34.802000+00:00,2f9e942b-5170-419f-989c-bc2c7e1c923b
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,7828e5f6-9cc7-4cc6-b20b-7bf844c38ccc,subscription,model_health,2024-06-06 06:04:08.111000+00:00,75d15cfc-3ded-41ff-b1bf-805ce134e6be
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,91a07f4c-50b6-426d-93da-94438a7edc9a,subscription,mrm,2024-07-02 10:04:06.083000+00:00,e130d4dc-5b57-4808-8cee-1f834b33f3e4
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,345f4646-871a-4e0c-b30f-abcf9a83ea71,subscription,model_health,2024-07-05 13:57:02.336000+00:00,37464a51-58b9-4620-afe6-d710be384daa
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,1ae2f751-e220-48a9-b049-d2404311f314,subscription,drift,2024-07-15 11:31:15.872000+00:00,dc52c272-b69c-47c3-abb6-c0bb4e1cfe4d
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,95e480af-2fec-43ef-b34e-b3a24572b35d,subscription,model_health,2024-07-11 09:30:12.810000+00:00,b53585b2-a86b-4d53-b4ec-7e3be95e1739
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,91a07f4c-50b6-426d-93da-94438a7edc9a,subscription,model_health,2024-06-02 08:26:53.650000+00:00,2c69e8d5-09bc-4c69-8e98-e63ff5a5098d


Note: First 10 records were displayed.


In [59]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id




 Waiting for end of monitor instance creation 3c7dfc4b-d9c8-4802-a03d-ba91b92d5256 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'3c7dfc4b-d9c8-4802-a03d-ba91b92d5256'

### Drift configuration

####  Note: you can choose to enable/disable (True or False) model or data drift within config

In [60]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [61]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.1,
    "train_drift_model": True,
    "enable_model_drift": False,
    "enable_data_drift": True
}


drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id




 Waiting for end of monitor instance creation b9a3b5d4-16bc-400c-9e6c-0e9f593101c7 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'b9a3b5d4-16bc-400c-9e6c-0e9f593101c7'

#### Finding feature importances

This is done because we will deduce the important and most important features to the model. This will help in narrowing down the analysis of Drift evaluation in the UI

In [62]:
from sklearn.inspection import permutation_importance

result = permutation_importance(risk_model, train_data.drop('Risk', axis=1), train_data.Risk, random_state=0)
result = np.mean(np.abs(result.importances), axis=1)
fields = [col for col in train_data.columns if col != "Risk"]

feature_importance = dict(zip(fields, result))


### Drift V2 Configuration

In [63]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift_v2" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift v2 monitor instance with id: ', monitor_instance.metadata.id)

In [64]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
        "min_samples": 10,
        "max_samples": 1000,
        "train_archive": True,
        "features": {
            "fields": fields,
            "importances": feature_importance
        }
    }
drift_v2_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

drift_v2_monitor_instance_id = drift_v2_monitor_details.metadata.id
drift_v2_monitor_instance_id

'c61a2e92-9218-430a-96b2-0d6262fba0ae'

## Score the model again now that monitoring is configured

This next section randomly selects 200 records from the data feed and sends those records to the model for predictions. This is enough to exceed the minimum threshold for records set in the previous section, which allows OpenScale to begin calculating fairness.

In [65]:
from IPython.utils import io
!rm german_credit_feed.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_feed.json
!ls -lh german_credit_feed.json

rm: german_credit_feed.json: No such file or directory
-rw-r--r--  1 nelwin  staff   2.9M Jul 31 12:00 german_credit_feed.json


Score 200 randomly chosen records

In [66]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(200):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

scoring_response = wml_client.deployments.score(deployment_uid, payload_scoring)

time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)

if pl_records_count == 8:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": scoring_response['predictions'][0]['fields'], "values":scoring_response['predictions'][0]['values']},
                   response_time=460
               )])
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

**Note:** Now in payload table should be total 208 records.

In [67]:
print("Number of records in payload table:", wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id))

Number of records in payload table:  208


## Run fairness monitor

Kick off a fairness monitor run on current data. The monitor runs hourly, but can be manually initiated using the Python client, the REST API, or the graphical user interface.

In [68]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 3c7dfc4b-d9c8-4802-a03d-ba91b92d5256 




finished

---------------------------
 Successfully finished run 
---------------------------




In [69]:
time.sleep(10)

wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

## Run drift monitor


Kick off a drift monitor run on current data. The monitor runs three hours, but can be manually initiated using the Python client, the REST API.

In [70]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 31c96d8a-9d38-4695-825e-466e7265cb71 




finished

---------------------------
 Successfully finished run 
---------------------------




In [71]:
time.sleep(5)

wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2024-07-16 06:19:17.267002+00:00,data_drift_magnitude,8d86edf2-07fd-4a9f-b625-11c838d21e02,0.125,None,0.1,[],drift,b9a3b5d4-16bc-400c-9e6c-0e9f593101c7,31c96d8a-9d38-4695-825e-466e7265cb71,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68


## Run drift v2 monitor


Kick off a drift v2 monitor run on current data. The monitor runs every day, but can be manually initiated using the Python client, the REST API.

In [72]:
drift_v2_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_v2_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9 




running................................................
finished

---------------------------
 Successfully finished run 
---------------------------




In [73]:
time.sleep(15)

wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_v2_monitor_instance_id)

2024-07-16 06:19:52.904904+00:00,records_processed,b7e32f25-be24-4583-95b3-bf0c52109ffb,208.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,c61a2e92-9218-430a-96b2-0d6262fba0ae,0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:19:52.904904+00:00,confidence_drift_score,b7e32f25-be24-4583-95b3-bf0c52109ffb,0.1862,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,c61a2e92-9218-430a-96b2-0d6262fba0ae,0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:19:52.904904+00:00,records_processed,b7e32f25-be24-4583-95b3-bf0c52109ffb,208.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,c61a2e92-9218-430a-96b2-0d6262fba0ae,0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:19:52.904904+00:00,confidence_drift_score,b7e32f25-be24-4583-95b3-bf0c52109ffb,0.1868,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,c61a2e92-9218-430a-96b2-0d6262fba0ae,0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:19:58.188821+00:00,records_processed,e314eb41-daf5-4bc4-bf92-c56cccba4eb7,208.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,c61a2e92-9218-430a-96b2-0d6262fba0ae,0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:19:58.188821+00:00,confidence_drift_score,e314eb41-daf5-4bc4-bf92-c56cccba4eb7,0.1657,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,c61a2e92-9218-430a-96b2-0d6262fba0ae,0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:19:58.188821+00:00,records_processed,e314eb41-daf5-4bc4-bf92-c56cccba4eb7,208.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,c61a2e92-9218-430a-96b2-0d6262fba0ae,0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:19:58.188821+00:00,confidence_drift_score,e314eb41-daf5-4bc4-bf92-c56cccba4eb7,0.1667,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,c61a2e92-9218-430a-96b2-0d6262fba0ae,0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:19:58.596595+00:00,records_processed,75c7061d-8b95-4f94-bfc7-9c4b8d354476,208.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:predictedLabel']",drift_v2,c61a2e92-9218-430a-96b2-0d6262fba0ae,0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68
2024-07-16 06:19:58.596595+00:00,prediction_drift_score,75c7061d-8b95-4f94-bfc7-9c4b8d354476,0.0469,None,0.05,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:predictedLabel']",drift_v2,c61a2e92-9218-430a-96b2-0d6262fba0ae,0fd0b8bc-944d-49f0-a71d-7bf2ec7098c9,subscription,f1a960ef-b474-43f9-8621-d0b1f2a4ef68


Note: First 10 records were displayed.


## Configure Explainability

Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [74]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation 7a3a484f-20b8-45e7-ad3c-ee1b02d760cf 




preparing..
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Run explanation for sample record

In [75]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result
print(result)

Running explanations on scoring IDs: ['7512f474-6c88-44bd-a591-24350774b1ad-1']
{
  "metadata": {
    "explanation_task_ids": [
      "2377ff91-77c7-441a-b7c8-b70a48ab0a59"
    ],
    "created_by": "IBMid-662005298W",
    "created_at": "2024-07-16T06:24:59.486239Z"
  }
}


# Custom monitors and metrics <a name="custom"></a>

## Register custom monitor

In [76]:
def get_definition(monitor_name):
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    
    for definition in monitor_definitions:
        if monitor_name == definition.entity.name:
            return definition
    
    return None

In [77]:
monitor_name = 'my model performance'
metrics = [MonitorMetricRequest(name='sensitivity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8)]),
          MonitorMetricRequest(name='specificity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.75)])]
tags = [MonitorTagRequest(name='region', description='customer geographical region')]

existing_definition = get_definition(monitor_name)

if existing_definition is None:
    custom_monitor_details = wos_client.monitor_definitions.add(name=monitor_name, metrics=metrics, tags=tags, background_mode=False).result
else:
    custom_monitor_details = existing_definition




 Waiting for end of adding monitor definition my_model_performance 




finished

-------------------------------------------------
 Successfully finished adding monitor definition 
-------------------------------------------------




### Show available monitors types

In [78]:
wos_client.monitor_definitions.show()

my_model_performance,my model performance,"['sensitivity', 'specificity']"
fairness,Fairness,"['Fairness value', 'Average odds difference', 'False discovery rate difference', 'Error rate difference', 'False negative rate difference', 'False omission rate difference', 'False positive rate difference', 'True positive rate difference', 'Average absolute odds difference', 'Statistical parity difference', 'Impact score']"
model_health,Model health,"['Total scoring requests', 'Total records', 'Average records', 'Median records', 'Maximum records', 'Minimum records', 'Total payload size', 'Average payload size', 'Median payload size', 'Minimum payload size', 'Maximum payload size', 'Average API throughput', 'Minimum API throughput', 'Maximum API throughput', 'Median API throughput', 'Average API latency', 'Minimum API latency', 'Maximum API latency', 'Median API latency', 'Average record throughput', 'Minimum record throughput', 'Maximum record throughput', 'Median record throughput', 'Average record latency', 'Minimum record latency', 'Maximum record latency', 'Median record latency', 'Users', 'Errors', 'Data errors', 'System errors', 'Total input token count', 'Average input token count', 'Median input token count', 'Maximum input token count', 'Minimum input token count', 'Total output token count', 'Average output token count', 'Median output token count', 'Maximum output token count', 'Minimum output token count']"
data_health,Data Health,"['Absence Count', 'Empty Strings', 'Data Type Mismatch', 'Class Confusion', 'Duplicate Rows', 'Unique Columns']"
performance,Performance,['Number of records']
explainability,Explainability,['Global explanation stability']
mrm,Model risk management monitoring,"['Tests run', 'Tests passed', 'Tests failed', 'Tests skipped', 'Fairness score', 'Quality score', 'Drift score']"
generative_ai_quality,Generative AI Quality,"['Records processed', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-Lsum', 'SARI', 'METEOR', 'F1 Score', 'Precision', 'Recall', 'BLEU', 'Jaccard similarity', 'Cosine similarity', 'Output data PII', 'Input data PII', 'Output data HAP', 'Input data HAP', 'Readability', 'Micro F1 Score', 'Macro F1 Score', 'Micro precision', 'Macro precision', 'Micro recall', 'Macro recall', 'Exact match', 'Faithfulness', 'Context relevance', 'Answer relevance', 'Unsuccessful requests']"
drift_v2,Drift v2,"['Output drift', 'Prediction drift', 'Model quality drift', 'Feature drift', 'Input metadata drift', 'Output metadata drift', 'Records processed', 'New records processed']"
drift,Drift,"['Drop in accuracy', 'Predicted accuracy', 'Drop in data consistency']"


Note: First 10 records were displayed.


### Get monitors uids and details

In [79]:
custom_monitor_id = custom_monitor_details.metadata.id

print(custom_monitor_id)

my_model_performance


In [80]:
custom_monitor_details = wos_client.monitor_definitions.get(monitor_definition_id=custom_monitor_id).result
print('Monitor definition details:', custom_monitor_details)

Monitor definition details: {
  "metadata": {
    "id": "my_model_performance",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:monitor_definition:my_model_performance",
    "url": "/v2/monitor_definitions/my_model_performance",
    "created_at": "2024-07-16T06:25:10.852000Z",
    "created_by": "IBMid-662005298W"
  },
  "entity": {
    "name": "my model performance",
    "metrics": [
      {
        "name": "sensitivity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.8
          }
        ],
        "expected_direction": "increasing",
        "id": "sensitivity"
      },
      {
        "name": "specificity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.75
          }
        ],
        "expected_direction": "increasing",
        "id": "specificity"
      }
    ],
    "tags": [
      {
        "name": "region

## Enable custom monitor for subscription

In [81]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
    )

thresholds = [MetricThresholdOverride(metric_id='sensitivity', type = MetricThresholdTypes.LOWER_LIMIT, value=0.9)]

custom_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=custom_monitor_id,
            parameters = {},
            target=target
).result




 Waiting for end of monitor instance creation 75dd6aa9-1c17-431e-bc14-019feee5592d 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Get monitor instance id and configuration details

In [82]:
custom_monitor_instance_id = custom_monitor_instance_details.metadata.id

In [83]:
custom_monitor_instance_details = wos_client.monitor_instances.get(custom_monitor_instance_id).result
print(custom_monitor_instance_details)

{
  "metadata": {
    "id": "75dd6aa9-1c17-431e-bc14-019feee5592d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:monitor_instance:75dd6aa9-1c17-431e-bc14-019feee5592d",
    "url": "/v2/monitor_instances/75dd6aa9-1c17-431e-bc14-019feee5592d",
    "created_at": "2024-07-16T06:25:46.655000Z",
    "created_by": "IBMid-662005298W",
    "modified_at": "2024-07-16T06:25:46.786000Z",
    "modified_by": "IBMid-662005298W"
  },
  "entity": {
    "data_mart_id": "80e6093f-5acf-4eb7-9da6-7ba9bf56a929",
    "monitor_definition_id": "my_model_performance",
    "target": {
      "target_type": "subscription",
      "target_id": "f1a960ef-b474-43f9-8621-d0b1f2a4ef68"
    },
    "parameters": {},
    "status": {
      "state": "active"
    }
  }
}


## Storing custom metrics

In [84]:
from datetime import datetime, timezone, timedelta
import uuid
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
custom_monitoring_run_id = str(uuid.uuid4())
measurement_request = [MonitorMeasurementRequest(timestamp=datetime.now(timezone.utc), 
                                                 metrics=[{"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}], run_id=custom_monitoring_run_id)]
print(measurement_request[0])

{
  "timestamp": "2024-07-16T06:25:53.155818Z",
  "metrics": [
    {
      "specificity": 0.78,
      "sensitivity": 0.67,
      "region": "us-south"
    }
  ]
}


In [85]:
published_measurement_response = wos_client.monitor_instances.measurements.add(
    monitor_instance_id=custom_monitor_instance_id,
    monitor_measurement_request=measurement_request).result
published_measurement_id = published_measurement_response[0]["measurement_id"]
print(published_measurement_response)

[{'measurement_id': '43acd6d2-8847-4cc7-bc0d-59c22d0932aa', 'metrics': [{'region': 'us-south', 'sensitivity': 0.67, 'specificity': 0.78}], 'timestamp': '2024-07-16T06:25:53.155818Z'}]


### List and get custom metrics

In [86]:
time.sleep(5)
published_measurement = wos_client.monitor_instances.measurements.get(monitor_instance_id=custom_monitor_instance_id, measurement_id=published_measurement_id).result
print(published_measurement)

{
  "metadata": {
    "id": "43acd6d2-8847-4cc7-bc0d-59c22d0932aa",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:measurement:43acd6d2-8847-4cc7-bc0d-59c22d0932aa",
    "url": "/v2/monitor_instances/75dd6aa9-1c17-431e-bc14-019feee5592d/measurements/43acd6d2-8847-4cc7-bc0d-59c22d0932aa",
    "created_at": "2024-07-16T06:25:56.829050Z",
    "created_by": "IBMid-662005298W"
  },
  "entity": {
    "timestamp": "2024-07-16T06:25:53.155818Z",
    "values": [
      {
        "metrics": [
          {
            "id": "sensitivity",
            "value": 0.67
          },
          {
            "id": "specificity",
            "value": 0.78
          }
        ],
        "tags": [
          {
            "id": "region",
            "value": "us-south"
          }
        ]
      }
    ],
    "issue_count": 0,
    "target": {
      "target_type": "subscription",
      "target_id": "f1a960ef-b474-43f9-8621-d0b1f2a4e

## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [87]:
wos_client.data_sets.show_records(payload_data_set_id, limit=5)

None,630,7512f474-6c88-44bd-a591-24350774b1ad-1,1.0,6.0,1.0,2,less_100,2024-07-16T06:18:47.201Z,0.8866628721250828,"[1.0, 3.0, 6.0, 0.0, 3.0, 1.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 10.0, 630.0, 2.0, 2.0, 33.0, 1.0, 1.0]",less_0,vacation,No Risk,0.0,0.0,unskilled,bank,0.0,3.0,none,10,33,all_credits_paid_back,2.0,2,0.0,3.0,1.0,0.0,yes,rent,1.0,1,5b644de6-5b83-4930-ac85-c600b2c0ac4f,"[17.733257442501653, 2.2667425574983446]",none,0.0,female,1,savings_insurance,less_1,"[0.8866628721250828, 0.11333712787491725]"
None,1845,7512f474-6c88-44bd-a591-24350774b1ad-10,0.0,9.0,3.0,2,100_to_500,2024-07-16T06:18:47.201Z,0.8184510088196267,"[3.0, 2.0, 9.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 22.0, 1845.0, 2.0, 2.0, 31.0, 1.0, 1.0]",greater_200,business,No Risk,0.0,0.0,skilled,none,1.0,2.0,none,22,31,outstanding_credit,0.0,2,0.0,1.0,0.0,1.0,no,rent,1.0,1,5b644de6-5b83-4930-ac85-c600b2c0ac4f,"[16.369020176392535, 3.6309798236074644]",none,0.0,male,1,savings_insurance,4_to_7,"[0.8184510088196267, 0.1815489911803732]"
None,3180,7512f474-6c88-44bd-a591-24350774b1ad-100,0.0,0.0,0.0,2,500_to_1000,2024-07-16T06:18:47.201Z,0.707344075368278,"[0.0, 1.0, 0.0, 2.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 9.0, 3180.0, 2.0, 4.0, 31.0, 1.0, 1.0]",no_checking,car_new,No Risk,1.0,0.0,unskilled,stores,2.0,1.0,none,9,31,credits_paid_to_date,1.0,4,0.0,1.0,1.0,0.0,yes,own,0.0,1,5b644de6-5b83-4930-ac85-c600b2c0ac4f,"[14.146881507365562, 5.853118492634437]",none,0.0,male,1,car_other,4_to_7,"[0.707344075368278, 0.29265592463172185]"
None,311,7512f474-6c88-44bd-a591-24350774b1ad-101,1.0,4.0,3.0,3,500_to_1000,2024-07-16T06:18:47.201Z,0.7874175935565438,"[3.0, 1.0, 4.0, 2.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 10.0, 311.0, 3.0, 2.0, 36.0, 1.0, 1.0]",greater_200,appliances,No Risk,1.0,0.0,skilled,none,2.0,1.0,none,10,36,credits_paid_to_date,0.0,2,0.0,1.0,0.0,0.0,yes,rent,1.0,1,5b644de6-5b83-4930-ac85-c600b2c0ac4f,"[15.748351871130874, 4.251648128869124]",none,0.0,female,1,car_other,4_to_7,"[0.7874175935565438, 0.21258240644345622]"
None,9300,7512f474-6c88-44bd-a591-24350774b1ad-102,0.0,9.0,2.0,5,500_to_1000,2024-07-16T06:18:47.201Z,0.578602331772885,"[2.0, 1.0, 9.0, 2.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 2.0, 1.0, 1.0, 25.0, 9300.0, 5.0, 3.0, 46.0, 3.0, 1.0]",0_to_200,business,Risk,1.0,1.0,management_self-employed,none,2.0,1.0,co-applicant,25,46,credits_paid_to_date,0.0,3,1.0,1.0,2.0,1.0,no,own,0.0,3,5b644de6-5b83-4930-ac85-c600b2c0ac4f,"[8.427953364542299, 11.5720466354577]",yes,1.0,male,1,car_other,4_to_7,"[0.42139766822711494, 0.578602331772885]"


## Insert historical fairness metrics

In [88]:
historyDays = 7

In [89]:
!rm history_fairness_v2.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_fairness_v2.json
!ls -lh history_fairness_v2.json

rm: history_fairness_v2.json: No such file or directory
-rw-r--r--  1 nelwin  staff   1.1M Jul 31 13:53 history_fairness_v2.json


In [90]:
from datetime import datetime, timedelta, timezone

from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Source
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Measurements
with open("history_fairness_v2.json") as f:
    fairness_values = json.load(f)
    for day in range(historyDays):
        print('Loading day', day + 1)
        daily_measurement_requests = []
        sources_list = []
        for hour in range(24):
            score_time = (datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
            index = (day * 24 + hour) % len(fairness_values) # wrap around and reuse values if needed
            fairness_values[index]["timestamp"] = score_time
            #print(score_time) 
            fairness_value = fairness_values[index]
            metrics_list = fairness_value["metrics"]
            sources = fairness_value["sources"]
            sources_list = []
            for source in sources:
                source_id = source["id"]
                source_type = source["type"]
                source_data = source["data"]
                if source_id == "bias_detection_summary":
                    source_data["evaluated_at"] = score_time
                    source_data["favourable_class"] = ["No Risk"]
                    source_data["unfavourable_class"] = ["Risk"]
                    source_data["score_type"] = "disparate impact"
                sources_list.append(
                    Source(
                        id=source_id,
                        type=source_type,
                        data=source_data
                    )
                )  
            measurement_request = MonitorMeasurementRequest(metrics=metrics_list, sources=sources_list, timestamp=score_time)
            daily_measurement_requests.append(measurement_request)
        measurements_client = Measurements(wos_client)
        measurements_client.add(monitor_instance_id=fairness_monitor_instance_id, monitor_measurement_request=daily_measurement_requests)     
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical debias metrics

In [91]:
!rm history_debias_v2.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_debias_v2.json
!ls -lh history_debias_v2.json

-rw-r--r--  1 nelwin  staff   725K Jul 31 14:16 history_debias_v2.json


In [92]:
with open("history_debias_v2.json") as f:
    debias_values = json.load(f)
    for day in range(historyDays):
        print('Loading day', day + 1)
        daily_measurement_requests = []
        sources_list = []
        for hour in range(24):
            score_time = (datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
            index = (day * 24 + hour) % len(debias_values) # wrap around and reuse values if needed
            debias_values[index]["timestamp"] = score_time
            debias_value = debias_values[index]
            metrics_list = debias_value["metrics"]
            sources = debias_value["sources"]
            sources_list = []
            for source in sources:
                sources_list.append(
                    Source(
                        id=source["id"],
                        type=source["type"],
                        data=source["data"]
                    )
                )  
            measurement_request = MonitorMeasurementRequest(metrics=metrics_list, sources=sources_list, timestamp=score_time)
            daily_measurement_requests.append(measurement_request)
        measurements_client = Measurements(wos_client)
        measurements_client.add(monitor_instance_id=fairness_monitor_instance_id, monitor_measurement_request=daily_measurement_requests)         

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical quality metrics

In [93]:
measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    quality_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        
        metric = {"area_under_roc": measurements[day]}
                
        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [metric])
        quality_measurement_requests.append(measurement_request)
        
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=quality_monitor_instance_id,
                                            monitor_measurement_request=quality_measurement_requests).result    
    
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical confusion matrixes

In [94]:
!rm history_quality_metrics.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_quality_metrics.json
!ls -lh history_quality_metrics.json

-rw-r--r--  1 nelwin  staff    78K Jul 31 14:16 history_quality_metrics.json


In [95]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Source

with open('history_quality_metrics.json') as json_file:
    records = json.load(json_file)
    
for day in range(historyDays):
    index = 0
    cm_measurement_requests = []
    print('Loading day', day + 1)
    
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"

        metric = records[index]['metrics']
        source = records[index]['sources']

        
        measurement_request = {"timestamp": score_time, "metrics": [metric], "sources": [source]}
        cm_measurement_requests.append(measurement_request)

        index+=1

    response = wos_client.monitor_instances.measurements.add(monitor_instance_id=quality_monitor_instance_id, monitor_measurement_request=cm_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical performance metrics

In [96]:
target = Target(
        target_type=TargetTypes.INSTANCE,
        target_id=payload_data_set_id
    )


performance_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=wos_client.monitor_definitions.MONITORS.PERFORMANCE.ID,
            target=target
).result
performance_monitor_instance_id = performance_monitor_instance_details.metadata.id




 Waiting for end of monitor instance creation 924ec8e9-266a-4b46-8f61-5975e1929b2a 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [97]:
for day in range(historyDays):
    performance_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        score_count = random.randint(60, 600)
        
        metric = {"record_count": score_count, "data_set_type": "scoring_payload"}
        
        measurement_request = {"timestamp": score_time, "metrics": [metric]}
        performance_measurement_requests.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=performance_monitor_instance_id,
                                            monitor_measurement_request=performance_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical drift measurements

In [98]:
!rm history_drift_measurement_*.json
with io.capture_output() as captured_0:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_0.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_1.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_2.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_3.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_4.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_5.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_6.json
!ls -lh history_drift_measurement_*.json

-rw-r--r--  1 nelwin  staff   831K Jul 31 14:17 history_drift_measurement_0.json
-rw-r--r--  1 nelwin  staff   867K Jul 31 14:17 history_drift_measurement_1.json
-rw-r--r--  1 nelwin  staff   870K Jul 31 14:17 history_drift_measurement_2.json
-rw-r--r--  1 nelwin  staff   909K Jul 31 14:17 history_drift_measurement_3.json
-rw-r--r--  1 nelwin  staff   840K Jul 31 14:17 history_drift_measurement_4.json
-rw-r--r--  1 nelwin  staff   835K Jul 31 14:17 history_drift_measurement_5.json
-rw-r--r--  1 nelwin  staff   840K Jul 31 14:17 history_drift_measurement_6.json


In [99]:
for day in range(historyDays):
    drift_measurements = []

    with open("history_drift_measurement_{}.json".format(day), 'r') as history_file:
        drift_daily_measurements = json.load(history_file)
    print('Loading day', day + 1)

    #Historical data contains 8 records per day - each represents 3 hour drift window.
    
    for nb_window, records in enumerate(drift_daily_measurements):
        for record in records:
            window_start =  datetime.utcnow() + timedelta(hours=(-(24 * day + (nb_window+1)*3 + 1))) # first_payload_record_timestamp_in_window (oldest)
            window_end = datetime.utcnow() + timedelta(hours=(-(24 * day + nb_window*3 + 1)))# last_payload_record_timestamp_in_window (most recent)
            #modify start and end time for each record
            record['sources'][0]['data']['start'] = window_start.isoformat() + "Z"
            record['sources'][0]['data']['end'] = window_end.isoformat() + "Z"
            
            
            metric = record['metrics'][0]
            source = record['sources'][0]

            measurement_request = {"timestamp": window_start.isoformat() + "Z", "metrics": [metric], "sources": [source]}
            
            drift_measurements.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=drift_monitor_instance_id,
                                            monitor_measurement_request=drift_measurements).result    

    
    print("Daily loading finished.")

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


In [100]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)

Datamart: 80e6093f-5acf-4eb7-9da6-7ba9bf56a929
Model: ce0b71a4-ecee-4e0d-b51a-123e0fad4acd
Deployment: db664cd4-2467-4975-b314-e24115c37ea5


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

## Next steps

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and can be deleted if necessary.